# Heart Rate (Variability) Feature Merging

This notebook merges different heart rate (variability) data computed into one long-format dataframe. The different data to be merged are:
* Mean heart rate per subject, MIST phase, and subphase (in bpm)
* *Normalized* Mean heart rate per subject, MIST phase and subphase (in percent relative to the global baseline $BL_{Glo}$)
* Heart rate variability measures per subject, MIST phase, and subphase
* *Percentage* of HR(V) measures per subject, MIST phase and subphase that are above the mean value during the global baseline $BL_{Glo}$

## Imports and Helper Functions

In [35]:
import re
from pathlib import Path

import pandas as pd
import numpy as np
import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns

import biopsykit as bp

%load_ext autoreload
%autoreload 2
%matplotlib widget

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
plt.close("all")

palette = bp.colors.fau_palette
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams['figure.figsize'] = (10, 5)
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['mathtext.default'] = "regular"

palette

[(0.0, 0.2196078431372549, 0.396078431372549),
 (0.596078431372549, 0.6431372549019608, 0.6823529411764706),
 (0.788235294117647, 0.5764705882352941, 0.07450980392156863),
 (0.0, 0.6941176470588235, 0.9215686274509803),
 (0.0, 0.6078431372549019, 0.4666666666666667),
 (0.5529411764705883, 0.0784313725490196, 0.1607843137254902)]

## Import Data

In [37]:
base_path = Path("../../../data/processed")
ecg_path = base_path.joinpath("ecg")

In [38]:
hr_data = bp.io.load_wide_format_csv(
    ecg_path.joinpath("cft_hr_mean.csv")
)

hr_data_norm = bp.io.load_wide_format_csv(
    ecg_path.joinpath("cft_hr_mean_normalized.csv")
)

hrv_data = bp.io.load_wide_format_csv(
    ecg_path.joinpath("cft_hrv_subphases.csv"), 
    index_cols=["condition", "subject", "phase", "subphase"]
)

hrv_above_baseline = bp.io.load_wide_format_csv(
    ecg_path.joinpath("cft_hrv_above_baseline.csv"), 
    index_cols=["condition", "subject", "phase", "subphase"]
)

## Merge Data

### Select Columns

In [41]:
hrv_data = hrv_data[["HRV_SDNN", "HRV_RMSSD", "HRV_SD1SD2", "HRV_pNN50", "HRV_pNN20"]]

### Rename Columns

In [42]:
hr_data.columns = ["HR"]
hr_data_norm.columns = ["HR_Norm"]

### Concat Dataframes, Reorder Index Levels

In [43]:
data_merged = pd.concat([hr_data, hr_data_norm, hrv_data, hrv_above_baseline], axis=1)
#data_merged.index = data_merged.index.set_names("name", level=-1)
#data_merged = data_merged.reset_index().set_index(["condition", "subject", "phase", "subphase", "type", "name"])
data_merged.columns.name = "name"
data_merged = pd.concat({"data": data_merged.stack()}, axis=1)
data_merged

data
condition subject phase subphase name                            
CFT       Vp01    MIST1 AT       HR                     92.039986
                                 HR_Norm                -4.866102
                                 HRV_SDNN               53.390947
                                 HRV_RMSSD              32.343869
                                 HRV_SD1SD2              0.318272
...                                                           ...
Control   Vp33    MIST3 RP_CFI   Time_BL_Glo_HRV_SDNN   35.658915
                                 Time_BL_Glo_HRV_RMSSD  31.782946
                                 Time_BL_Glo_HRV_pNN50  71.317829
                                 Time_BL_Glo_HRV_pNN20  67.441860
                                 Time_BL_Glo_HR         47.286822

[4032 rows x 1 columns]

## Export Data

In [44]:
data_merged.to_csv(ecg_path.joinpath("cft_measures_merged.csv"))